In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 3,
    "ProductFamily": 4,
    "ProductOffering": 5,
    "Facility": 6,
    "Parts": 7,
    "Warehouse": 8,
    "Supplier": 9,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000/api"
# API_URL = "http://172.17.149.238/api"

version = "L_10T_V2"

In [4]:
# create_ops, graph = generator.create_network(custom_counts)

# print(f"Generated {len(create_ops)} operations")

# from networkx.readwrite import json_graph
# print(json_graph.node_link_data(graph[0], edges="links"))

# direct_create_payload = {
#     "version": version,
#     "action": "direct_create",
#     "type": "schema",
#     "timestamp": create_ops[0]["timestamp"],
#     "payload": json_graph.node_link_data(graph[0], edges="links")
# }

# requests.post(f"{API_URL}/schema/live/update", json=direct_create_payload)


In [5]:
create_ops, graph = generator.create_network(custom_counts)

bulk_create_payload = {
    "version": version,
    "action": "bulk_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": []
}

for op in create_ops:
    bulk_create_payload["payload"].append(op["payload"])

print(bulk_create_payload)

requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

{'version': 'L_10T_V2', 'action': 'bulk_create', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': '1', 'node_type': 'Parts', 'properties': {'id': '1', 'name': 'Name_4582', 'description': 'Description_1348', 'type': 'Type B', 'cost': 3556.65, 'importance': 3, 'expected_life': 2962, 'units_in_chain': 20, 'expiry': 281}}, {'node_id': '1-1', 'node_type': 'Facility', 'properties': {'id': '1-1', 'name': 'Name_1260', 'type': 'Type C', 'location': 'Los Angeles', 'max_capacity': 3505, 'operating_cost': 2397.41}}, {'source_id': '1', 'target_id': '1-1', 'edge_type': 'PartsToFacility', 'properties': {'quantity': 214, 'distance_from_warehouse': 75.33, 'transport_cost': 606.23, 'lead_time': 1}}, {'node_id': '1-1-1', 'node_type': 'ProductOffering', 'properties': {'id': '1-1-1', 'name': 'Name_6503', 'cost': 2979.46, 'demand': 623}}, {'source_id': '1-1', 'target_id': '1-1-1', 'edge_type': 'FacilityToProductOfferings', 'properties': {'product_cost': 121.72, 'lead_time': 29, 'quantity_produced':

<Response [200]>

In [6]:
update_cycles = 10
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

Generated 19 update operations
{'version': 'L_10T_V2', 'action': 'bulk_update', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': 'Warehouse_3', 'node_type': 'Warehouse', 'payload': {'id': 'Warehouse_3', 'name': 'Name_1606', 'type': 'Type C', 'size': 'Medium', 'location': 'New York', 'max_capacity': 4394, 'current_capacity': 1527, 'safety_stock': 882}}, {'node_id': '4', 'node_type': 'Parts', 'payload': {'id': '4', 'name': 'Name_1464', 'description': 'Description_2794', 'type': 'Type C', 'cost': 2742.11, 'importance': 3, 'expected_life': 857, 'units_in_chain': 23, 'expiry': 299}}, {'node_id': '1', 'node_type': 'Parts', 'payload': {'id': '1', 'name': 'Name_4582', 'description': 'Description_7223', 'type': 'Type B', 'cost': 3556.65, 'importance': 2, 'expected_life': 2962, 'units_in_chain': 76, 'expiry': 281}}, {'node_id': 'Warehouse_8', 'node_type': 'Warehouse', 'payload': {'id': 'Warehouse_8', 'name': 'Name_7863', 'type': 'Type A', 'size': 'Extra Large', 'location': 'Houston', 'm

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)

In [8]:
# schema = json.load(open("../../../metadata/relations.json", "r"))
# generator = RandomNetworkGenerator(schema)

# custom_counts = {
#     "BusinessUnit": 1,
#     "ProductFamily": 2,
#     "ProductOffering": 3,
#     "Facility": 4,
#     "Parts": 5,
#     "Warehouse": 6,
#     "Supplier": 7,
# }

# # API_URL = "https://bug-hardy-obviously.ngrok-free.app"
# API_URL = "http://localhost:8000"

# version = "test181"

# create_ops, graph = generator.create_network(custom_counts)

# print(f"Generated {len(create_ops)} operations")

# from networkx.readwrite import json_graph
# print(json_graph.node_link_data(graph[0], edges="links"))

# import os
# import sys
# sys.path.append("../../../")
# from workers import direct_create

# direct_create(payload=json_graph.node_link_data(graph[0], edges="links"), timestamp = create_ops[0]["timestamp"], version = version)
# print("Direct create complete")

# update_cycles = 5
# for _ in range(update_cycles):
#     update_ops = generator.generate_updates(
#         node_updates=10,
#         edge_updates=10
#     )
#     print(f"Generated {len(update_ops)} update operations")

#     bulk_update_payload = {
#         "version": version,
#         "action": "bulk_update",
#         "type": "schema",
#         "timestamp": update_ops[0]["timestamp"],
#         "payload": []
#     }

#     for op in update_ops:
#         bulk_update_payload["payload"].append(op["payload"])

#     print(bulk_update_payload)

#     requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
#     time.sleep(1)